In [1]:
from  keras.datasets import imdb
from keras import models,layers
import numpy as np
from keras import optimizers
from keras import preprocessing
import os
import keras
keras.__version__

d:\programdata\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.2.4'

In [2]:
imdb_dir='F:\\study\\ml\\DataSet\\aclImdb'
train_dir=os.path.join(imdb_dir,'train')

In [3]:
labels=[]
texts=[]

for label_type in ['pos','neg']:
    dir_name = os.path.join(train_dir,label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:]=='.txt':
            f=open(os.path.join(dir_name,fname),encoding='utf-8')
            texts.append(f.read())
            f.close()
            if label_type == 'pos':
                labels.append(1)
            else:
                labels.append(0)
            


In [4]:
from keras.preprocessing.text import Tokenizer

In [5]:
maxlen=100
train_samples=10000
validation_samples=5000
max_words=10000

tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)
word_index=tokenizer.word_index

In [6]:
type(sequences)

list

In [7]:
sequences[0]

[309,
 6,
 3,
 1069,
 209,
 9,
 2161,
 30,
 1,
 169,
 55,
 14,
 46,
 82,
 5844,
 41,
 392,
 110,
 138,
 14,
 5340,
 58,
 4449,
 150,
 8,
 1,
 4988,
 5924,
 482,
 69,
 5,
 261,
 12,
 2002,
 6,
 73,
 2425,
 5,
 632,
 71,
 6,
 5340,
 1,
 5,
 2003,
 1,
 5925,
 1534,
 34,
 67,
 64,
 205,
 140,
 65,
 1230,
 1,
 4,
 1,
 223,
 901,
 29,
 3022,
 69,
 4,
 1,
 5845,
 10,
 693,
 2,
 65,
 1534,
 51,
 10,
 216,
 1,
 387,
 8,
 60,
 3,
 1467,
 3712,
 800,
 5,
 3513,
 177,
 1,
 392,
 10,
 1237,
 30,
 309,
 3,
 353,
 344,
 2974,
 143,
 130,
 5,
 7799,
 28,
 4,
 126,
 5340,
 1467,
 2373,
 5,
 309,
 10,
 532,
 12,
 108,
 1468,
 4,
 58,
 555,
 101,
 12,
 309,
 6,
 227,
 4174,
 48,
 3,
 2232,
 12,
 9,
 215]

In [8]:
data=preprocessing.sequence.pad_sequences(sequences,maxlen=maxlen)
labels=np.asarray(labels)
print('shape of data tensor',data.shape)
print('shape of label tensor',labels.shape)

shape of data tensor (25000, 100)
shape of label tensor (25000,)


In [9]:
indices=np.arange(data.shape[0])

In [10]:
np.random.shuffle(indices)

In [11]:
indices

array([20426,  8932, 16408, ...,  3434, 14325, 23271])

In [12]:
indices=np.arange(data.shape[0])
np.random.shuffle(indices)
data=data[indices]
labels=labels[indices]

In [13]:
data.shape

(25000, 100)

In [14]:
x_train=data[:train_samples]
y_train=labels[:train_samples]
x_val=data[train_samples:train_samples+validation_samples]
y_val=labels[train_samples:train_samples+validation_samples]

In [15]:
x_train.shape

(10000, 100)

In [16]:
x_val.shape

(5000, 100)

In [17]:
x_train.shape

(10000, 100)

In [18]:
glove_dir='F:\\study\\ml\\DataSet\\glove.6B'

In [19]:
embeddings_index={}
f=open(os.path.join(glove_dir,'glove.6B.100d.txt'),encoding='utf-8')
for line in f:
    values=line.split()
    word = values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeddings_index[word]=coefs
f.close()

In [20]:
len(embeddings_index)

400000

In [21]:
len(word_index)

88582

In [22]:
# x_train=data[:200]
# y_train=labels[:200]
# x_val=data[200:200+validation_samples]
# y_val=labels[200:200+validation_samples]

In [23]:
embeddings_dim=100
embeddings_matrix=np.zeros((max_words,embeddings_dim))
for word,i in word_index.items():
    if i < max_words:
        embeddings_vector=embeddings_index.get(word)
        if embeddings_vector is not None:
            embeddings_matrix[i]=embeddings_vector

In [24]:
model=models.Sequential()
model.add(layers.Embedding(max_words,embeddings_dim,input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.layers[0].set_weights([embeddings_matrix])
model.layers[0].trainable=False

In [26]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

In [27]:
x_train.shape

(10000, 100)

In [28]:
x_val.shape

(5000, 100)

In [29]:
history=model.fit(x_train,y_train,
                 epochs=10,
                 batch_size=32,
                 validation_data=(x_val,y_val))

Train on 10000 samples, validate on 5000 samples
Epoch 1/10
10000/10000 [==============================] - 2s 204us/step - loss: 0.6920 - acc: 0.5797 - val_loss: 0.6433 - val_acc: 0.6432
Epoch 2/10
10000/10000 [==============================] - 2s 194us/step - loss: 0.5973 - acc: 0.7053 - val_loss: 0.8031 - val_acc: 0.5542
Epoch 3/10
10000/10000 [==============================] - 2s 198us/step - loss: 0.5354 - acc: 0.7419 - val_loss: 0.5912 - val_acc: 0.7022
Epoch 4/10
10000/10000 [==============================] - 2s 195us/step - loss: 0.4390 - acc: 0.7989 - val_loss: 0.5816 - val_acc: 0.7146
Epoch 5/10
10000/10000 [==============================] - 2s 195us/step - loss: 0.3770 - acc: 0.8317 - val_loss: 0.6956 - val_acc: 0.6748
Epoch 6/10
10000/10000 [==============================] - 2s 196us/step - loss: 0.3274 - acc: 0.8590 - val_loss: 0.8192 - val_acc: 0.6650
Epoch 7/10
10000/10000 [==============================] - 2s 197us/step - loss: 0.2730 - acc: 0.8825 - val_loss: 0.7436 - v

######## no use pre embedding

In [30]:
model2=models.Sequential()
model2.add(layers.Embedding(max_words,embeddings_dim,input_length=maxlen))
model2.add(layers.Flatten())
model2.add(layers.Dense(32,activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 320,065
Non-trainable params: 1,000,000
_________________________________________________________________


In [31]:
model2.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

In [32]:
history2=model2.fit(x_train,y_train,
                 epochs=10,
                 batch_size=32,
                 validation_data=(x_val,y_val))

Train on 10000 samples, validate on 5000 samples
Epoch 1/10
10000/10000 [==============================] - 6s 594us/step - loss: 0.5075 - acc: 0.7315 - val_loss: 0.3977 - val_acc: 0.8216
Epoch 2/10
10000/10000 [==============================] - 6s 586us/step - loss: 0.1290 - acc: 0.9555 - val_loss: 0.4718 - val_acc: 0.8084
Epoch 3/10
10000/10000 [==============================] - 6s 595us/step - loss: 0.0095 - acc: 0.9978 - val_loss: 0.6775 - val_acc: 0.8136
Epoch 4/10
10000/10000 [==============================] - 6s 590us/step - loss: 5.2370e-04 - acc: 0.9998 - val_loss: 0.7587 - val_acc: 0.8192
Epoch 5/10
10000/10000 [==============================] - 6s 594us/step - loss: 7.6568e-06 - acc: 1.0000 - val_loss: 0.9066 - val_acc: 0.8210
Epoch 6/10
10000/10000 [==============================] - 6s 587us/step - loss: 1.7436e-06 - acc: 1.0000 - val_loss: 0.9699 - val_acc: 0.8236
Epoch 7/10
10000/10000 [==============================] - 6s 615us/step - loss: 1.3236e-07 - acc: 1.0000 - val_

In [33]:
test_dir=os.path.join(imdb_dir,'test')
test_labels=[]
test_texts=[]

for label_type in ['pos','neg']:
    dir_name = os.path.join(test_dir,label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:]=='.txt':
            f=open(os.path.join(dir_name,fname),encoding='utf-8')
            test_texts.append(f.read())
            f.close()
            if label_type == 'pos':
                test_labels.append(1)
            else:
                test_labels.append(0)
            


In [35]:
test_sequences=tokenizer.texts_to_sequences(test_texts)
x_test=preprocessing.sequence.pad_sequences(test_sequences,maxlen=maxlen)
y_test=np.asarray(test_labels)

In [36]:
model.evaluate(x_test,y_test)

25000/25000 [==============================] - 1s 52us/step


[0.9071477563762664, 0.69096]